In [11]:
import trimet_plan_game_helper_scripts as helpers
import geopandas as gpd
import folium
import seaborn as sns

In [12]:
tm_boundary = gpd.read_file("gis/export/tm_route_buffer_bounds.geojson")
trimet_crs = "EPSG:2913"

In [15]:
gdf_points, dist_btw_points = helpers.generate_points_within_tm_boundary(tm_boundary, trimet_crs)
gdf_points['points_str'] = gdf_points['points'].apply(lambda x: f"{round(x.y,6)}, {round(x.x,6)}")
gdf_points.explore(tiles='CartoDB positron')

In [16]:
fromplace, toplace = gdf_points['points_str'].to_numpy()
json_content = helpers.call_planner(fromplace, toplace)
itineraries_df = helpers.get_itinerary_paths(json_content)

itineraries_gdf = gpd.GeoDataFrame(itineraries_df, crs="4326", geometry="legGeometry")

unique_combinations = itineraries_df[itineraries_df['route_id']!='WALK'].groupby('itin_idx').agg(route_id_list=('route_id',list)).reset_index().drop_duplicates(subset='route_id_list')
unique_combinations['route_id_combo'] = unique_combinations['route_id_list'].apply(lambda x: " to ".join(x))

itineraries_reduced = itineraries_gdf.merge(unique_combinations, how='inner', on='itin_idx')

itineraries_centroid = itineraries_reduced.unary_union.convex_hull.centroid

# Get Unique continents
color_labels = itineraries_reduced['route_id'].unique()

# List of colors in the color palettes
hex_values = sns.color_palette("Set2", len(color_labels)).as_hex()

# Map continents to the colors
color_map = dict(zip(color_labels, hex_values))

m = folium.Map(location=[itineraries_centroid.y, itineraries_centroid.x], zoom_start=13, tiles='CartoDB positron')
origin_dest_points = folium.GeoJson(gdf_points.to_json(), name='origin_destination')
origin_dest_points.add_to(m)

for idx, itin_group in itineraries_reduced.groupby('itin_idx'):
    itin_idx = itin_group.iloc[0]['itin_idx']
    itin_route_combo = itin_group.iloc[0]['route_id_combo']
    group_json = itin_group.to_json()
    folium.GeoJson(group_json, name=f'{itin_route_combo}'
                   , tooltip=folium.GeoJsonTooltip(fields=['route_id'])
                   ,style_function=lambda feature: {
                                    'color': color_map[feature['properties']['route_id']],
                                    'weight': 1
                                }
                   ).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m